In [1]:
import pandas as pd
import csv
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.utils import shuffle
import os

In [2]:
l1 = 'eng'
l2 = 'ita'

conf = 7

In [3]:
l1_freq_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/MULTI/freq/{}'.format(l1)
l2_freq_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/MULTI/freq/{}'.format(l2)

pl_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/MULTI/filter/{0}_{1}.tsv'.format(l1, l2)

train_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/MULTI/train/{0}_{1}.tsv'.format(l1, l2)
test_fn = '/mnt/permanent/home/eszti/dipterv/panlex/data/panlex/tsv/MULTI/test/{0}_{1}.tsv'.format(l1, l2)

In [4]:
# Read l1 freq info

l1_ls = []
with open(l1_freq_fn) as f:
    lines = f.read().splitlines()
    for line in lines:
        l1_ls.append(line)

In [5]:
# 2519427
l1_f = pd.DataFrame({'eng' : l1_ls})
l1_f = l1_f.drop_duplicates(subset=('eng'))
l1_f['freq'] = l1_f.index
l1_f.head()
len(l1_f.index)

,eng,freq
0,",",0
1,.,1
2,the,2
3,</s>,3
4,of,4


2519395

In [6]:
# Read l2 freq info

l2_ls = []
with open(l2_freq_fn) as f:
    lines = f.read().splitlines()
    for line in lines:
        l2_ls.append(line)

In [7]:
# 985649
l2_f = pd.DataFrame({'ita' : l2_ls})
l2_f = l2_f.drop_duplicates(subset=('ita'))
l2_f['freq'] = l2_f.index
l2_f.head()
len(l2_f.index)

,ita,freq
0,</s>,0
1,",",1
2,.,2
3,-,3
4,',4


871053

In [8]:
# Read panlex data
pl = pd.read_csv(pl_fn, sep='\t', header=None, names = [l1, l2, 'score'])
pl['score'] = pl.score.astype(int)
pl.head()
len(pl.index)

,eng,ita,score
0,dirham,dirham,9
1,dinar,dinaro,9
2,dong,dong,9
3,lari,lari,9
4,dobra,dobra,9


187601

In [9]:
# Filter conf

pl = pl[pl.score >= conf]
len(pl.index)

69623

In [10]:
# Merge l1 freq info
pl_f = pd.merge(pl, l1_f, on=[l1])
pl_f.head()
len(pl_f.index)

,eng,ita,score,freq
0,dirham,dirham,9,132514
1,dinar,dinaro,9,43568
2,dinar,denaro,7,43568
3,dong,dong,9,7645
4,dong,uccello,7,7645


69623

In [11]:
# Merge l2 freq info
pl_ff = pd.merge(pl_f, l2_f, on=[l2])
pl_ff.head()
len(pl_ff.index)

,eng,ita,score,freq_x,freq_y
0,dirham,dirham,9,132514,85750
1,dinar,dinaro,9,43568,57543
2,dinar,denaro,7,43568,2982
3,dough,denaro,7,20950,2982
4,money,denaro,7,1079,2982


69623

In [12]:
# Create bins from l1

bin1 = l1_f.iloc[1000:5000]
bin2 = l1_f.iloc[5000:20000]
bin3 = l1_f.iloc[20000:50000]
bin4 = l1_f.iloc[50000:100000]
bin5 = l1_f.iloc[100000:200000]

In [13]:
def get_300_uq(binN, alle, N, no_dup=True):
    print('Number of words in {0}: {1}'.format(N, len(binN.index)))
    
    alle_bin = pd.merge(alle, binN, on=[l1])
    print('len alle_bin: {0}'.format(len(alle_bin.index)))
        
    if no_dup:
        alle_bin = alle_bin.drop_duplicates(subset=[l1], keep=False)
        print('len alle_bin nodup: {0}'.format(len(alle_bin.index)))
    
    alle_bin = shuffle(alle_bin)
    
    testN = alle_bin.head(300)
    print('len testN: {0}'.format(len(testN.index)))

    return testN

In [14]:
test1 = get_300_uq(bin1, pl_ff, 1)

Number of words in 1: 4000
len alle_bin: 13257
len alle_bin nodup: 616
len testN: 300


In [15]:
test2 = get_300_uq(bin2, pl_ff, 2)

Number of words in 2: 15000
len alle_bin: 21100
len alle_bin nodup: 2350
len testN: 300


In [16]:
test3 = get_300_uq(bin3, pl_ff, 3)

Number of words in 3: 30000
len alle_bin: 15559
len alle_bin nodup: 3889
len testN: 300


In [17]:
test4 = get_300_uq(bin4, pl_ff, 4)

Number of words in 4: 50000
len alle_bin: 8797
len alle_bin nodup: 3522
len testN: 300


In [18]:
test5 = get_300_uq(bin5, pl_ff, 5)

Number of words in 5: 100000
len alle_bin: 5200
len alle_bin nodup: 2777
len testN: 300


In [19]:
# Concat test

frames = [test1, test2, test3, test4, test5]
pl_test = pd.concat(frames)
pl_test.head()
len(pl_test.index)

,eng,ita,score,freq_x,freq_y,freq
12855,twelve,dodici,7,2795,2760,2795
12146,stated,dichiarato,7,1029,2582,1029
10114,massive,massiccio,7,3514,4775,3514
12971,premier,premier,7,1781,5102,1781
12938,sets,insiemistica,7,2461,131053,2461


1500

In [20]:
# Create train without duplicates

test_eng = pl_test[[l1]].copy()
len(test_eng.index)

pl_train = pl_ff[~pl_ff.eng.isin(set(test_eng.eng))]
len(pl_ff.index)
len(pl_train.index)

pl_train = pl_train.sort_values(['freq_x', 'freq_y'], ascending=True)
pl_train_nodup = pl_train.drop_duplicates(subset=[l1], keep=False)
len(pl_train_nodup.index)

pl_train_nodup_5K = pl_train_nodup.head(5000)
pl_train_nodup_5K.head(10)
pl_train_nodup_5K.tail(10)
len(pl_train_nodup_5K.index)

1500

69623

68123

11777

,eng,ita,score,freq_x,freq_y
4479,-,-,8,5,3
5030,made,fatto,7,61,159
4146,%,%,8,66,216
31855,should,dovere,7,68,7667
67973,two,due,7,69,57
46313,would,volere,7,73,5286
4772,years,anni,8,92,73
53070,these,questi,7,122,237
61350,september,settembre,7,132,195
61349,october,ottobre,7,135,220


,eng,ita,score,freq_x,freq_y
35050,obtuse,ottuso,7,39367,43795
46987,ganglia,gangli,7,39375,54245
39218,bedside,capezzale,7,39383,47406
52640,parsley,prezzemolo,7,39386,41092
61397,macrophages,macrofagi,7,39388,42966
59269,blinding,accecamento,7,39403,121075
61140,lymphocytes,linfociti,7,39410,20652
67218,macroeconomic,macroeconomico,7,39411,156142
68601,hyena,iena,7,39417,45708
64463,quadrilateral,quadrilatero,7,39426,31543


5000

In [21]:
# With duplicates

pl_train_nodup = pl_train
len(pl_train_nodup.index)

pl_train_nodup_5K = pl_train_nodup.head(5000)
pl_train_nodup_5K.head(10)
pl_train_nodup_5K.tail(10)
len(pl_train_nodup_5K.index)

68123

,eng,ita,score,freq_x,freq_y
10413,",",",",8,0,1
10415,",",.,8,0,2
4480,",",-,7,0,3
10412,.,",",8,1,1
10414,.,.,8,1,2
43627,the,il,7,2,9
43628,the,la,7,2,10
43619,the,i,7,2,25
43620,the,le,7,2,28
43623,the,gli,7,2,47


,eng,ita,score,freq_x,freq_y
8132,despite,con,7,844,24
29594,despite,nonostante,7,844,482
29590,despite,sebbene,7,844,1116
29585,despite,malgrado,7,844,3954
29599,despite,offesa,7,844,14025
9505,despite,insulto,7,844,20414
9527,despite,affronto,7,844,43592
9491,despite,oltraggio,7,844,45179
1911,read,leggere,7,845,1839
3432,read,indicare,7,845,2190


5000

In [22]:
eng_te = set(pl_test[l1])
eng_tr = set(pl_train_nodup_5K[l1])
len(eng_te & eng_tr)

spa_te = set(pl_test[l2])
spa_tr = set(pl_train_nodup_5K[l2])
len(spa_te & spa_tr)

0

127

In [30]:
def save_tsv(df, fn):
    if not os.path.exists(os.path.dirname(fn)):
        os.makedirs(os.path.dirname(fn))
    pd.DataFrame.to_csv(df, columns=[l1, l2], header=False, path_or_buf=fn, sep='\t', index=False)

In [33]:
save_tsv(pl_train_nodup_5K, train_fn)
save_tsv(pl_test, test_fn)